---
**Libraries**

In [ ]:
import numpy as np
import tensorflow as tf
from tensorflow import keras
from keras.utils import load_img
import warnings

import pandas as pd

#for testing
from IPython.display import Image

# for reading rgb for each pixel
#   seems to be different Images
from PIL import Image

import array as arr

# Mount google drive
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


<br>

---
**Get Training and Test Data for Random Forests**
<br><br>

This cell read in data from a csv containing the keypoints, bounding boxes, and label for each image. Labels describe the motion of the person in the image. In this case only the label and the keypoints are extracted. At the end, you see data being splitted into training and testing.

In [ ]:
dataset=pd.read_csv('/content/drive/Shareddrives/Team56_FallDetection/fall-01-cam0-rgb-20221014T162818Z-001/fall-01-cam0-rgb/test.csv')

#Setup training and testing data
dataset['label']=(dataset['label'].map({-1:2, 0:0, 1:1}))#Switch -1 to 2 for classifier

x=dataset['keypoints'].apply(eval).apply(list).values.tolist()
x=np.array([np.array(xi) for xi in x])
y=dataset['label'].values
y=np.array([np.array(xi) for xi in y])

x_train=x
y_train=y

In [ ]:
dataset=pd.read_csv('/content/drive/Shareddrives/Team56_FallDetection/fall-01-cam0-rgb-20221014T162818Z-001/fall-01-cam0-rgb/test2.csv')

#Setup training and testing data
dataset['label']=(dataset['label'].map({-1:2, 0:0, 1:1}))#Switch -1 to 2 for classifier

x=dataset['keypoints'].apply(eval).apply(list).values.tolist()
x=np.array([np.array(xi) for xi in x])
y=dataset['label'].values
y=np.array([np.array(xi) for xi in y])


x_test=x
y_test=y

<br>

---
**Train Random Forest Model**
<br><br>

The cell below is the Random Forest, which fit the random forest model to the training data, so it could be trained.

The model is evaluated through testing it with the testing data, which in this case is x_test, the keypoint data.

The evaluation is done with the classification report function.

In [ ]:
import pandas as pd
import numpy as np
from sklearn.ensemble import RandomForestClassifier
clf = RandomForestClassifier(max_depth=2, random_state=0)
from ast import literal_eval
from sklearn.metrics import classification_report


# Train and Test
clf.fit(list(x_train), y_train)
y_pred=clf.predict(list(x_test))

print(classification_report(y_test, y_pred))
y_pred_RF=y_pred

              precision    recall  f1-score   support

           0       0.88      0.79      0.83        28
           1       1.00      1.00      1.00        41
           2       0.86      0.92      0.89        39

    accuracy                           0.92       108
   macro avg       0.91      0.90      0.91       108
weighted avg       0.92      0.92      0.92       108



<br>

---
**Get Dataset for CNN**

In [ ]:
# import required module
import os
 
# assign directory
directory = '/content/drive/Shareddrives/Team56_FallDetection/fall-01-cam0-rgb-20221014T162818Z-001/fall-01-cam0-rgb/fall-01-cam0-rgb'

files=[]

# iterate over files in
# that directory
for filename in os.scandir(directory):
    if filename.is_file():
        
        files.append(filename.path)
files.sort()
(len(files))

154

<br>

---
**(Function) Extract RGB Values**

In [ ]:
def rgb_extract(file):
  path = file


  im = Image.open(path)
  pix = im.load()               # pixel access object that can be manipulated

 
  im_size = [im.width, im.height]


  print("actual rgb at (639, 479) = {0}".format(pix[639,479]))               # print RGBA value of image


  # create 2D empty array [[y]x]
  rgb_vals = [[0] * im.height] * im.width

  for a in range(0, im.width):
    for b in range(0, im.height):
   
      
      rgb_vals[a][b] = pix[a,b]
     
  return np.array(rgb_vals)

<br>

---
**Extract rgb data from training set**

In [ ]:
#Extract rgb files from each images
import numpy as np 
a = []
for file in files:
  a.append(rgb_extract(file))
a = np.asarray(a)

actual rgb at (639, 479) = (69, 85, 84)
actual rgb at (639, 479) = (62, 87, 84)
actual rgb at (639, 479) = (60, 84, 86)
actual rgb at (639, 479) = (70, 86, 86)
actual rgb at (639, 479) = (72, 86, 95)
actual rgb at (639, 479) = (69, 83, 92)
actual rgb at (639, 479) = (69, 88, 82)
actual rgb at (639, 479) = (71, 86, 91)
actual rgb at (639, 479) = (70, 83, 89)
actual rgb at (639, 479) = (71, 87, 87)
actual rgb at (639, 479) = (71, 86, 93)
actual rgb at (639, 479) = (71, 87, 86)
actual rgb at (639, 479) = (70, 84, 85)
actual rgb at (639, 479) = (69, 82, 88)
actual rgb at (639, 479) = (72, 87, 90)
actual rgb at (639, 479) = (71, 82, 84)
actual rgb at (639, 479) = (72, 83, 87)
actual rgb at (639, 479) = (74, 86, 86)
actual rgb at (639, 479) = (70, 84, 87)
actual rgb at (639, 479) = (68, 84, 84)
actual rgb at (639, 479) = (72, 88, 85)
actual rgb at (639, 479) = (70, 86, 86)
actual rgb at (639, 479) = (67, 85, 89)
actual rgb at (639, 479) = (65, 86, 87)
actual rgb at (639, 479) = (72, 83, 85)


<br>

---
**Obtain labels and bounding box from dataset**

In [ ]:
#Gather data for CNN.  Use image and bounding boxes for 2d cnn
import pandas as pd
import numpy as np
'''
Read data set, use 'test.csv' is you cannot mount from drive, otherwise use '/content/drive/Shareddrives/Team56_FallDetection/fall-01-cam0-rgb-20221014T162818Z-001/fall-01-cam0-rgb/test.csv'

'''
file='/content/drive/Shareddrives/Team56_FallDetection/fall-01-cam0-rgb-20221014T162818Z-001/fall-01-cam0-rgb/test.csv'#'/content/drive/MyDrive/fall-01-cam0-rgb/test.csv'#Use for when drive is mounted
#file='test.csv'#use for uploaded files
dataset=pd.read_csv(file)

#Change the labels so there are not any negatives
dataset['label']=(dataset['label'].map({-1:2, 0:0, 1:1}))#Change -1 to 2

#Reformat data

x=a #X is the image data
box=dataset['box'].apply(eval).apply(list).values.tolist()#Bounding box data
y=dataset['label'].values
y=np.array([np.array(xi) for xi in y])#Y is the label data


In [ ]:
import numpy as np
import tensorflow as tf
from tensorflow import keras
from keras import layers

In [ ]:
from sklearn.model_selection import train_test_split
indices=range(154)

xtrain=x
ytrain=y
xboxtrain=box
xboxtrain = np.asarray(xboxtrain)
yboxtrain=y


---
CNN Model - Get Training Data

In [ ]:
#Get list of image files names
# import required module
import os
 
# assign directory
directory = '/content/drive/Shareddrives/Team56_FallDetection/fall-01-cam0-rgb-20221014T162818Z-001/fall-01-cam0-rgb/fall-02-cam0-rgb'

files=[]

# iterate over files in
# that directory
for filename in os.scandir(directory):
    if filename.is_file():
        
        files.append(filename.path)
files.sort()
(len(files))



#Extract rgb files from each images
import numpy as np 
a = []
for file in files:
  a.append(rgb_extract(file))
a = np.asarray(a)



#Gather data for CNN.  Use image and bounding boxes for 2d cnn
import pandas as pd
import numpy as np


file='/content/drive/Shareddrives/Team56_FallDetection/fall-01-cam0-rgb-20221014T162818Z-001/fall-01-cam0-rgb/test2.csv'
#file='test.csv'#use for uploaded files
dataset=pd.read_csv(file)

#Change the labels so there are not any negatives
dataset['label']=(dataset['label'].map({-1:2, 0:0, 1:1}))#Change -1 to 2

#Reformat data

x=a #X is the image data
box=dataset['box'].apply(eval).apply(list).values.tolist()#Bounding box data
y=dataset['label'].values
y=np.array([np.array(xi) for xi in y])#Y is the label data




actual rgb at (639, 479) = (76, 87, 89)
actual rgb at (639, 479) = (75, 85, 86)
actual rgb at (639, 479) = (70, 83, 89)
actual rgb at (639, 479) = (74, 85, 89)
actual rgb at (639, 479) = (75, 86, 88)
actual rgb at (639, 479) = (67, 79, 75)
actual rgb at (639, 479) = (77, 86, 85)
actual rgb at (639, 479) = (73, 85, 83)
actual rgb at (639, 479) = (77, 87, 89)
actual rgb at (639, 479) = (74, 85, 91)
actual rgb at (639, 479) = (74, 83, 88)
actual rgb at (639, 479) = (63, 77, 80)
actual rgb at (639, 479) = (72, 86, 86)
actual rgb at (639, 479) = (75, 87, 87)
actual rgb at (639, 479) = (75, 89, 92)
actual rgb at (639, 479) = (70, 85, 88)
actual rgb at (639, 479) = (76, 87, 89)
actual rgb at (639, 479) = (75, 86, 88)
actual rgb at (639, 479) = (72, 87, 92)
actual rgb at (639, 479) = (74, 87, 96)
actual rgb at (639, 479) = (77, 87, 86)
actual rgb at (639, 479) = (75, 86, 90)
actual rgb at (639, 479) = (63, 81, 81)
actual rgb at (639, 479) = (65, 76, 78)
actual rgb at (639, 479) = (75, 86, 92)


In [ ]:
xtest=x
ytest=y
xboxtest=box
xboxtest = np.asarray(xboxtest)
yboxtest=y

---
CNN Model - Set Up

In [ ]:
import tensorflow
from tensorflow import keras
from keras.layers import Input, Conv2D, Dense, Activation, Flatten, concatenate, Normalization,MaxPooling2D,Dropout,Rescaling
from keras.models import Model

image_input = Input(( 640, 480, 3))
x=Rescaling(1./255),

x = Conv2D(640, kernel_size=8, strides=4,activation='relu')(image_input)
x=MaxPooling2D(pool_size=2, strides=2)(x)
x=Dropout(0.25)(x)
x = Flatten()(x)
x = Dense(3)(x)

vector_input = Input((4,))

y = Dense(3)(vector_input)

z = concatenate([x, y])



z = Dense(3)(z)
z = Activation('softmax')(z)

model = Model([image_input, vector_input], [z])
model.summary()

Model: "model"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_1 (InputLayer)           [(None, 640, 480, 3  0           []                               
                                )]                                                                
                                                                                                  
 conv2d (Conv2D)                (None, 159, 119, 64  123520      ['input_1[0][0]']                
                                0)                                                                
                                                                                                  
 max_pooling2d (MaxPooling2D)   (None, 79, 59, 640)  0           ['conv2d[0][0]']                 
                                                                                              

In [ ]:
model.compile(
    loss=keras.losses.SparseCategoricalCrossentropy(),#cross entropy
    optimizer=keras.optimizers.RMSprop(),
    metrics=["accuracy"],
)

In [ ]:
#normalize the data
v_min = xboxtrain.min(axis=(0, 1), keepdims=True)
v_max = xboxtrain.max(axis=(0, 1), keepdims=True)
xboxtrain=(xboxtrain - v_min)/(v_max - v_min)

v_min = xtrain.min(axis=(0, 1), keepdims=True)
v_max = xtrain.max(axis=(0, 1), keepdims=True)
xtrain=(xtrain - v_min)/(v_max - v_min)

v_min = xboxtest.min(axis=(0, 1), keepdims=True)
v_max = xboxtest.max(axis=(0, 1), keepdims=True)
xboxtest=(xboxtest - v_min)/(v_max - v_min)

v_min = xtest.min(axis=(0, 1), keepdims=True)
v_max = xtest.max(axis=(0, 1), keepdims=True)
xtest=(xtest - v_min)/(v_max - v_min)

---
CNN Model - Training

In [ ]:
epochs=1

history = model.fit(
  [xtrain,xboxtrain],
  ytrain,
  validation_split=0.20,
  epochs=epochs,
  verbose=2,
  shuffle=True,
)

4/4 - 43s - loss: 89.3880 - accuracy: 0.5447 - val_loss: 13.0342 - val_accuracy: 0.0000e+00 - 43s/epoch - 11s/step


In [ ]:
model.save('/content/drive/Shareddrives/Team56_FallDetection/saved_models/my_model_9_demo_seventh_try')


---
CNN Model - Load Model

In [ ]:
import tensorflow as tf
from tensorflow import keras

new_model = tf.keras.models.load_model('/content/drive/Shareddrives/Team56_FallDetection/saved_models/my_model_9_demo_seventh_try')

# Check its architecture
new_model.summary()
model=new_model

Model: "model_9"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_37 (InputLayer)          [(None, 640, 480, 3  0           []                               
                                )]                                                                
                                                                                                  
 conv2d_25 (Conv2D)             (None, 159, 119, 64  123520      ['input_37[0][0]']               
                                0)                                                                
                                                                                                  
 max_pooling2d_18 (MaxPooling2D  (None, 79, 59, 640)  0          ['conv2d_25[0][0]']              
 )                                                                                          

In [ ]:
y_pred_ohe =model.predict([xtest, xboxtest])
len(y_pred_ohe)

4/4 [==============================] - 10s 2s/step


108

In [ ]:
from sklearn.metrics import confusion_matrix
y_pred_labels = np.argmax(y_pred_ohe, axis=1)  

print( confusion_matrix(y_true=ytest, y_pred=y_pred_labels))
print(y_pred_labels)
print(ytest)

y_pred_CNN=y_pred_labels

[[12  0 16]
 [ 4  0 37]
 [17  0 22]]
[0 0 2 2 2 2 0 2 0 2 0 0 0 2 0 0 2 2 2 0 2 0 2 0 2 2 2 2 2 2 0 2 2 0 0 0 2
 0 2 0 2 0 0 2 0 0 2 0 2 0 2 2 0 0 0 0 2 2 2 0 2 2 2 2 2 2 2 2 2 2 2 2 2 2
 2 2 2 2 0 2 2 2 2 0 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 0 2 0 2 2 2 2]
[2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2
 2 2 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 1 1 1 1 1 1
 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1]


In [ ]:
from sklearn.metrics import classification_report
print(classification_report(ytest, y_pred_labels))

              precision    recall  f1-score   support

           0       0.36      0.43      0.39        28
           1       0.00      0.00      0.00        41
           2       0.29      0.56      0.39        39

    accuracy                           0.31       108
   macro avg       0.22      0.33      0.26       108
weighted avg       0.20      0.31      0.24       108



/usr/local/lib/python3.8/dist-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.8/dist-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.8/dist-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [ ]:
# overlay text
# Importing the PIL library
from PIL import Image
from PIL import ImageDraw
from PIL import ImageFont

def text_to_image(image_path,label,new_image_path):
# Open an Image
  img = Image.open(image_path)


  # Call draw Method to add 2D graphics in an image
  I1 = ImageDraw.Draw(img)
  
  # Custom font style and font size
  myFont = ImageFont.truetype('FreeMono.ttf', 65)
  
  # Add Text to an image
  I1.text((10, 10), label, font=myFont, fill =(255, 0, 0))
  
  # Display edited image
  img.show()
  
  # Save the edited image
  img.save(new_image_path+image_path[(image_path.rindex("/")+1):])

Compare results

In [ ]:
# import required module
import os
 
# assign directory
directory = '/content/drive/Shareddrives/Team56_FallDetection/fall-01-cam0-rgb-20221014T162818Z-001/fall-01-cam0-rgb/fall-02-cam0-rgb'#'/content/drive/MyDrive/fall-01-cam0-rgb/fall-01-cam0-rgb'

files=[]

# iterate over files in
# that directory
for filename in os.scandir(directory):
    if filename.is_file():
        
        files.append(filename.path)
files.sort()
(len(files))


print(y_pred_RF)
print(y_pred_CNN)
i=0
while i<len(y_pred_RF):
  if(y_pred_RF[i]==y_pred_CNN[i] and (y_pred_RF[i]==0)):
    print("yes")
    print(files[i])
    text_to_image(files[i],"Fall","/content/drive/Shareddrives/Team56_FallDetection/fall-01-cam0-rgb-20221014T162818Z-001/fall-01-cam0-rgb/fall-02-cam0-rgb_labelled/")
  else:
    text_to_image(files[i],"","/content/drive/Shareddrives/Team56_FallDetection/fall-01-cam0-rgb-20221014T162818Z-001/fall-01-cam0-rgb/fall-02-cam0-rgb_labelled/")
    
  i=i+1



[2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 0 2 2 2 2 2 0 2 2 2 2 2 2 2 2 2 2 0 2 2 2 2
 2 2 2 2 2 0 0 0 0 2 2 0 0 2 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 1 1 1 1 1 1
 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1]
[0 0 2 2 2 2 0 2 0 2 0 0 0 2 0 0 2 2 2 0 2 0 2 0 2 2 2 2 2 2 0 2 2 0 0 0 2
 0 2 0 2 0 0 2 0 0 2 0 2 0 2 2 0 0 0 0 2 2 2 0 2 2 2 2 2 2 2 2 2 2 2 2 2 2
 2 2 2 2 0 2 2 2 2 0 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 0 2 0 2 2 2 2]
yes
/content/drive/Shareddrives/Team56_FallDetection/fall-01-cam0-rgb-20221014T162818Z-001/fall-01-cam0-rgb/fall-02-cam0-rgb/fall-02-cam0-rgb-016.png
yes
/content/drive/Shareddrives/Team56_FallDetection/fall-01-cam0-rgb-20221014T162818Z-001/fall-01-cam0-rgb/fall-02-cam0-rgb/fall-02-cam0-rgb-022.png
yes
/content/drive/Shareddrives/Team56_FallDetection/fall-01-cam0-rgb-20221014T162818Z-001/fall-01-cam0-rgb/fall-02-cam0-rgb/fall-02-cam0-rgb-043.png
yes
/content/drive/Shareddrives/Team56_FallDetection/fall-01-cam0-rgb-20221014T162818Z-001/fall-01-cam0-rgb/fa